# `ipywidgets.interact` for logistic sigmoid and Bayes classifiers

This notebook shows how I used `ipywidgets.interact` to better understand equations in machine learning, including the logistic sigmoid and Bayes classifiers.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact

## Logistic Sigmoid

The sigmoid function shows up a lot in machine learning. For one input dimension, one form I frequently see is

$$\sigma(x) = \frac{1}{1 + \exp(-v^{\top}x + b)}$$

This code allows one to vary $v$ and $b$ for two sigmoid functions. The code produces something like:

![sigmoid example](2018-05-04-sigmoid.png)

In [2]:
def logistic_sigmoid(xx, vv, b):
    return 1 / (1 + np.exp(-(np.dot(vv, xx) + b)))

plt.clf()
grid_size = 0.01
x_grid = np.arange(-5, 5, grid_size)


def plot_logistic_sigmoid(vv1, bb1, vv2, bb2):
    plt.plot(x_grid, logistic_sigmoid(x_grid, vv=vv1, b=bb1), '-b')
    plt.plot(x_grid, logistic_sigmoid(x_grid, vv=vv2, b=bb2), '-r')
    plt.axis([-5, 5, -0.5, 1.5])
    plt.show()


interact(
    plot_logistic_sigmoid, 
    vv1=(-12, 10, .25), 
    bb1=(-10, 10), 
    vv2=(-10, 12), 
    bb2=(-10, 10)
)

interactive(children=(FloatSlider(value=-1.0, description='vv1', max=10.0, min=-12.0, step=0.25), IntSlider(value=0, description='bb1', max=10, min=-10), IntSlider(value=1, description='vv2', max=12, min=-10), IntSlider(value=0, description='bb2', max=10, min=-10), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_logistic_sigmoid>

## Bayes Classifiers

(These notes are from the machine learning course I took last semester. The relevant notes are here: [MLPR](http://www.inf.ed.ac.uk/teaching/courses/mlpr/2017/notes/w3a_intro_classification.html))

Bayes classifiers are a method of classifying used in machine learning. 

For each class $k$, the Bayes classifier has a prior, $P(y = k)$, and a distribution over features, $P(\textbf x \mid y = k)$, such as a Gaussian $\mathcal N (\textbf x; \hat\mu_k, \Sigma_k)$. 
The probability of the decision is $P(y=k \mid \textbf x) = \frac{\mathcal N(\textbf x; \hat \mu_k, \Sigma_k)}{\sum_k'\mathcal N(\textbf x; \hat \mu_k', \Sigma_k')}$. ([Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) is an example of a classifier that assumes features are independent, i.e. $\Sigma_k$ is diagonal.)

I wanted to see what shapes the decision surface could take on. 
If the covariances of the classes are equal, the decision boundary is linear.
Even with one input dimension, the decision boundaries do interesting things. For example, if one class's variance is larger than the other, then there are two decision boundaries! And the sigmoid function makes another appearance.

### One input dimension

This code allows you to adjust the parameters of a Bayes classifier with one input dimension and two classes. It produces something like

![Bayes classifer with one dimension](2018-05-04-bayes-1.png)


In [3]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

plt.clf()
grid_size = 0.1
x_grid = np.arange(-5, 5, grid_size)

def draw(zoom, pi_1, mu_1, mu_2, var_1, var_2):

    Z1 = multivariate_normal(mu_1, var_1)
    Z2 = multivariate_normal(mu_2, var_2)
    
    # Decision boundary!
    Z_num = pi_1 * Z1.pdf(x_grid)
    z_den = ((1 - pi_1) * Z2.pdf(x_grid) + pi_1 * Z1.pdf(x_grid)) + 1e-300
    Z = Z_num / z_den

    plt.figure()
    plt.plot(x_grid, Z, c='k')
    plt.plot(x_grid, Z1.pdf(x_grid))
    plt.plot(x_grid, Z2.pdf(x_grid))

    # set the axis based on the zoom
    plt.axis([-5, 5, -0.05, 1.05])
    plt.show()


interact(
    draw, 
    zoom=(1, 10),  # zoom into image
    pi_1=(0.01, 1),  # prior on first class
    mu_1=(-5, 12),  # mean of first class
    mu_2=(-12, 5),   # mean of second class
    var_1=(0, 2.),  # variance of first class
    var_2=(0, 2.),  # variance of second class
)

interactive(children=(IntSlider(value=5, description='zoom', max=10, min=1), FloatSlider(value=0.505, description='pi_1', max=1.0, min=0.01), IntSlider(value=3, description='mu_1', max=12, min=-5), IntSlider(value=-4, description='mu_2', max=5, min=-12), FloatSlider(value=1.0, description='var_1', max=2.0), FloatSlider(value=1.0, description='var_2', max=2.0), Output()), _dom_classes=('widget-interact',))

<function __main__.draw>

## Two input dimensions

This code produces something like

![Bayes classifier with 2 input dimensions](2018-05-04-bayes-2.png)

In [4]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal


delta = 0.2
x = np.arange(-60, 60, delta)
y = np.arange(-40, 40, delta)
X, Y = np.meshgrid(x, y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y


def draw(zoom, pi_1, m_1, m_2, stretch, stretch_2, cov_1, cov_2):
    Sigma_1 = np.array([[stretch, cov_1], [cov_1, 1.0]])
    mu_1 = np.array([m_1, 0.0])

    Sigma_2 = np.array([[stretch_2, cov_2], [cov_2, stretch_2]])
    mu_2 = np.array([m_2, 0.0])

    Z1 = multivariate_normal(mu_1, Sigma_1)
    Z2 = multivariate_normal(mu_2, Sigma_2)
    
    # compute the decision boundary!
    Z_num = pi_1 * Z1.pdf(pos)
    z_den = ((1 - pi_1) * Z2.pdf(pos) + pi_1 * Z1.pdf(pos)) + 1e-300  # add an offset to avoid divide by 0
    Z = Z_num / z_den - 0.5

    plt.figure()
    plt.contour(X, Y, Z, 10, colors='k')    
    plt.contour(X, Y, pi_1 * Z1.pdf(pos), 5)
    plt.contour(X, Y, (1 - pi_1) * Z2.pdf(pos), 5)

    # set the axis based on the zoom
    plt.axis([int(i / (zoom * 4)) for i in [-150, 150, -100, 100]])
    plt.show()


interact(
    draw, 
    zoom=(1, 10), 
    pi_1=(0.01, 1),  # prior on the first class
    m_1=(-5, 12),  # x dimension of the first class's mu
    m_2=(-12, 5),  # x dimension of the second class's mu
    stretch=(1, 5),  # x dimension variance
    stretch_2=(1, 5),   # x and y dimension variance
    cov_1=(0, 1.0),  # covariance between two dimension's for first class
    cov_2=(0, 1.0),  # covariance between two dimension's for second class
)

interactive(children=(IntSlider(value=5, description='zoom', max=10, min=1), FloatSlider(value=0.505, description='pi_1', max=1.0, min=0.01), IntSlider(value=3, description='m_1', max=12, min=-5), IntSlider(value=-4, description='m_2', max=5, min=-12), IntSlider(value=3, description='stretch', max=5, min=1), IntSlider(value=3, description='stretch_2', max=5, min=1), FloatSlider(value=0.5, description='cov_1', max=1.0), FloatSlider(value=0.5, description='cov_2', max=1.0), Output()), _dom_classes=('widget-interact',))

<function __main__.draw>